In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/graduate-admissions/Admission_Predict.csv
/kaggle/input/graduate-admissions/Admission_Predict_Ver1.1.csv


In [3]:
!pip install pyspark

     |████████████████████████████████| 212.3 MB 10 kB/s s eta 0:00:01    |████▋                           | 30.8 MB 15.0 MB/s eta 0:00:13     |█████████▋                      | 63.8 MB 35.7 MB/s eta 0:00:05     |███████████▎                    | 74.6 MB 35.7 MB/s eta 0:00:04     |████████████████▎               | 108.0 MB 33.8 MB/s eta 0:00:04
     |████████████████████████████████| 198 kB 51.4 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=79bf9a976586fd6b55c5b7096dc178ee88fedec19c2030b31a18505808a76707
  Stored in directory: /root/.cache/pip/wheels/43/47/42/bc413c760cf9d3f7b46ab7cd6590e8c47ebfd19a7386cd4a57
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [9]:
df = spark.read.csv('/kaggle/input/graduate-admissions/Admission_Predict_Ver1.1.csv', inferSchema=True, header=True)
df.show()

+----------+---------+-----------+-----------------+---+----+----+--------+----------------+
|Serial No.|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit |
+----------+---------+-----------+-----------------+---+----+----+--------+----------------+
|         1|      337|        118|                4|4.5| 4.5|9.65|       1|            0.92|
|         2|      324|        107|                4|4.0| 4.5|8.87|       1|            0.76|
|         3|      316|        104|                3|3.0| 3.5| 8.0|       1|            0.72|
|         4|      322|        110|                3|3.5| 2.5|8.67|       1|             0.8|
|         5|      314|        103|                2|2.0| 3.0|8.21|       0|            0.65|
|         6|      330|        115|                5|4.5| 3.0|9.34|       1|             0.9|
|         7|      321|        109|                3|3.0| 4.0| 8.2|       1|            0.75|
|         8|      308|        101|                2|3.0| 4.0| 7.9|    

In [10]:
df.printSchema()

root
 |-- Serial No.: integer (nullable = true)
 |-- GRE Score: integer (nullable = true)
 |-- TOEFL Score: integer (nullable = true)
 |-- University Rating: integer (nullable = true)
 |-- SOP: double (nullable = true)
 |-- LOR : double (nullable = true)
 |-- CGPA: double (nullable = true)
 |-- Research: integer (nullable = true)
 |-- Chance of Admit : double (nullable = true)



In [14]:
shape = (df.count(), len(df.columns))
print(shape)

(500, 9)


In [15]:
df.describe().show()

+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|summary|       Serial No.|         GRE Score|      TOEFL Score|University Rating|               SOP|              LOR |              CGPA|          Research|   Chance of Admit |
+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|  count|              500|               500|              500|              500|               500|               500|               500|               500|                500|
|   mean|            250.5|           316.472|          107.192|            3.114|             3.374|             3.484| 8.576440000000003|              0.56| 0.7217399999999996|
| stddev|144.4818327679989|11.295148372354712|6.081867659564538|1.143511800759815|0.9910036207566072|0.92

In [17]:
df = df.drop('Serial No.')
df.show()

+---------+-----------+-----------------+---+----+----+--------+----------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit |
+---------+-----------+-----------------+---+----+----+--------+----------------+
|      337|        118|                4|4.5| 4.5|9.65|       1|            0.92|
|      324|        107|                4|4.0| 4.5|8.87|       1|            0.76|
|      316|        104|                3|3.0| 3.5| 8.0|       1|            0.72|
|      322|        110|                3|3.5| 2.5|8.67|       1|             0.8|
|      314|        103|                2|2.0| 3.0|8.21|       0|            0.65|
|      330|        115|                5|4.5| 3.0|9.34|       1|             0.9|
|      321|        109|                3|3.0| 4.0| 8.2|       1|            0.75|
|      308|        101|                2|3.0| 4.0| 7.9|       0|            0.68|
|      302|        102|                1|2.0| 1.5| 8.0|       0|             0.5|
|      323|     

In [27]:
for i in df.columns:
    print(i + ":" ,df[df[i].isNull()].count())

GRE Score: 0
TOEFL Score: 0
University Rating: 0
SOP: 0
LOR : 0
CGPA: 0
Research: 0
Chance of Admit : 0


In [56]:
# I had an issue in the cell below this one because of the trailing space in the 'Chance of Admit' column name. Printed out the names of the columns to see that annoying space
df.schema.names

['GRE Score',
 'TOEFL Score',
 'University Rating',
 'SOP',
 'LOR ',
 'CGPA',
 'Research',
 'Chance of Admit ']

In [57]:
for col in df.columns:
    print(col + ":", df.stat.corr("Chance of Admit ", col))
    
# I will use only the first 3 most correlated columns as features

GRE Score: 0.8103506354632598
TOEFL Score: 0.7922276143050823
University Rating: 0.6901323687886892
SOP: 0.6841365241316723
LOR : 0.6453645135280112
CGPA: 0.882412574904574
Research: 0.5458710294711379
Chance of Admit : 1.0


In [58]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['GRE Score', 'TOEFL Score', 'CGPA'], outputCol='features')

In [60]:
output = assembler.transform(df)
output.show()

+---------+-----------+-----------------+---+----+----+--------+----------------+------------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit |          features|
+---------+-----------+-----------------+---+----+----+--------+----------------+------------------+
|      337|        118|                4|4.5| 4.5|9.65|       1|            0.92|[337.0,118.0,9.65]|
|      324|        107|                4|4.0| 4.5|8.87|       1|            0.76|[324.0,107.0,8.87]|
|      316|        104|                3|3.0| 3.5| 8.0|       1|            0.72| [316.0,104.0,8.0]|
|      322|        110|                3|3.5| 2.5|8.67|       1|             0.8|[322.0,110.0,8.67]|
|      314|        103|                2|2.0| 3.0|8.21|       0|            0.65|[314.0,103.0,8.21]|
|      330|        115|                5|4.5| 3.0|9.34|       1|             0.9|[330.0,115.0,9.34]|
|      321|        109|                3|3.0| 4.0| 8.2|       1|            0.75| [321.0,10

In [62]:
final_data = output.select('features', 'Chance of Admit ')
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Chance of Admit : double (nullable = true)



In [65]:
from pyspark.ml.regression import LinearRegression

train, test = final_data.randomSplit([0.8, 0.2])
lr = LinearRegression(featuresCol='features', labelCol='Chance of Admit ')
model= lr.fit(train)

In [66]:
print('Coefficients: ', model.coefficients)
print('Intercept: ', model.intercept)

Coefficients:  [0.0022425074230307063,0.003439962149560529,0.1439712191839732]
Intercept:  -1.590012225064237


In [67]:
summary = model.summary

In [68]:
print('RMSE ', summary.rootMeanSquaredError)

RMSE  0.06164971277307208


In [69]:
predictions = model.transform(test)

In [70]:
predictions.show()

+------------------+----------------+-------------------+
|          features|Chance of Admit |         prediction|
+------------------+----------------+-------------------+
| [295.0,99.0,7.57]|            0.37| 0.5019458467589912|
| [295.0,99.0,7.65]|            0.57| 0.5134635442937088|
| [296.0,95.0,7.54]|            0.44| 0.4861093690082605|
|[296.0,101.0,7.68]|             0.6| 0.5269051125913797|
|[297.0,101.0,7.67]|            0.57| 0.5277079078225706|
|  [298.0,98.0,7.5]|            0.44| 0.4951554215356444|
| [298.0,99.0,7.46]|            0.53|0.49283653491784585|
|[299.0,100.0,7.42]|            0.42| 0.4927601557230783|
|[299.0,100.0,7.89]|            0.59| 0.5604266287395456|
|  [300.0,97.0,8.1]|            0.65| 0.5825832057425291|
| [301.0,99.0,8.22]|            0.64| 0.6089821837667577|
|[301.0,106.0,8.47]|            0.57| 0.6690547236096747|
|[302.0,101.0,7.96]|            0.46| 0.5806720985010767|
| [302.0,110.0,8.5]|            0.65| 0.6893762162064665|
| [303.0,102.0